In [47]:
#워닝 메시지
import warnings
warnings.filterwarnings('ignore')


In [48]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [49]:
data= load_breast_cancer()
X_data = data.data
y_data = data.target
X_train,X_test,y_train,y_test = train_test_split(X_data,y_data, test_size=0.2,random_state=0)


In [50]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from lightgbm import LGBMClassifier

In [51]:
## 개별 모델들
svm= SVC()
rf = RandomForestClassifier(n_estimators=100, random_state=0)
lr= LogisticRegression()

## 메타 모델
lgbm= LGBMClassifier()

In [52]:
svm.fit(X_train,y_train)
rf.fit(X_train,y_train)
lr.fit(X_train,y_train)

LogisticRegression()

In [53]:
svm_pred= svm.predict(X_test)
rf_pred= rf.predict(X_test)
lr_pred= lr.predict(X_test)

print(accuracy_score(svm_pred,y_test))
print(accuracy_score(rf_pred,y_test))
print(accuracy_score(lr_pred,y_test))

0.9298245614035088
0.9649122807017544
0.9473684210526315


In [54]:
new_data = np.array([svm_pred,rf_pred,lr_pred])
new_data.shape

(3, 114)

In [55]:
new_data = new_data.T
new_data.shape

(114, 3)

In [56]:
new_data[:10]

array([[1, 0, 0],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1]])

In [57]:
lgbm.fit(new_data,y_test)
lgbm_pred = lgbm.predict(new_data)
print(accuracy_score(y_test, lgbm_pred))

0.9736842105263158


---
- CV

In [58]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error

def get_stacking_base_datasets(model, X_train_n, y_train_n, X_test_n, n_folds):
    kf = KFold(n_splits=n_folds, shuffle=False)
    
    # 추후 메타 모델이 사용할 학습 데이터 반환을 위한 넘파이 배열 초기화
    train_fold_pred = np.zeros((X_train_n.shape[0], 1))
    test_pred = np.zeros((X_test_n.shape[0], n_folds))
    print(model.__class__.__name__,' model 시작')
    
    for folder_counter, (train_index, valid_index) in enumerate(kf.split(X_train_n)):
        print('\t 폴드 세트: ',folder_counter+1,' 시작')
        X_tr = X_train_n[train_index]
        y_tr = y_train_n[train_index]
        X_te = X_train_n[valid_index]
        
        model.fit(X_tr, y_tr)
       
        train_fold_pred[valid_index, :] = model.predict(X_te).reshape(-1, 1)
        test_pred[:, folder_counter] = model.predict(X_test_n)
        
    # 폴드 세트 내에서 원본 테스트 데이터를 예측한 데이터를 평균하여 테스트 데이터로 생성
    test_pred_mean = np.mean(test_pred, axis=1).reshape(-1,1)
    
    return train_fold_pred, test_pred_mean

In [59]:
svm_train, svm_test = get_stacking_base_datasets(svm, X_train, y_train, X_test, 7)
rf_train, rf_test = get_stacking_base_datasets(rf, X_train, y_train, X_test, 7)
lr_train, lr_test = get_stacking_base_datasets(lr, X_train, y_train, X_test, 7)

SVC  model 시작
	 폴드 세트:  1  시작
	 폴드 세트:  2  시작
	 폴드 세트:  3  시작
	 폴드 세트:  4  시작
	 폴드 세트:  5  시작
	 폴드 세트:  6  시작
	 폴드 세트:  7  시작
RandomForestClassifier  model 시작
	 폴드 세트:  1  시작


	 폴드 세트:  2  시작
	 폴드 세트:  3  시작
	 폴드 세트:  4  시작
	 폴드 세트:  5  시작
	 폴드 세트:  6  시작
	 폴드 세트:  7  시작
LogisticRegression  model 시작
	 폴드 세트:  1  시작
	 폴드 세트:  2  시작
	 폴드 세트:  3  시작
	 폴드 세트:  4  시작
	 폴드 세트:  5  시작
	 폴드 세트:  6  시작
	 폴드 세트:  7  시작


In [60]:
Stack_final_X_train = np.concatenate((svm_train, rf_train, lr_train), axis=1)
Stack_final_X_test = np.concatenate((svm_test, rf_test, lr_test), axis=1)
print('원본 학습 피처 데이터 shape:', X_train.shape, '원본 테스트 피처 shape:',X_test.shape)
print('스태킹 학습 피처 데이터 shape:',Stack_final_X_train.shape,
     '스태킹 테스트 피처 데이터 shape:',Stack_final_X_test.shape)

원본 학습 피처 데이터 shape: (455, 30) 원본 테스트 피처 shape: (114, 30)
스태킹 학습 피처 데이터 shape: (455, 3) 스태킹 테스트 피처 데이터 shape: (114, 3)


#### 메타 모델 -->lgbm

In [61]:
lgbm_final= LGBMClassifier()

In [62]:
lgbm_final.fit(Stack_final_X_train, y_train)
stack_final = lgbm_final.predict(Stack_final_X_test)

print('최종 메타 모델의 예측 정확도: ', accuracy_score(y_test, stack_final))
print('최종 메타 모델의 예측 f1: ', f1_score(y_test, stack_final))

최종 메타 모델의 예측 정확도:  0.9736842105263158
최종 메타 모델의 예측 f1:  0.9777777777777777


---
- pipe - line을 이용한 stacking

In [63]:
# 필요한 라이브러리 임포트
import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.pipeline import Pipeline

# 데이터를 불러오기
data = load_breast_cancer()
X_data = data.data
y_data = data.target
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=0)

# 7-fold 교차 검증을 위한 KFold 객체 생성 (수정)
n_splits = 7
kf = KFold(n_splits=n_splits, shuffle=True)  # shuffle=True로 수정

# 첫 번째 레벨 모델 정의
models_first_level = [
    ('logreg', LogisticRegression()),
    ('svc', SVC()),  # XGBoost(XGB)를 SVC로 변경
    ('rf', RandomForestClassifier())
]

# 메타 모델로 사용할 LightGBM 정의
meta_model = LGBMClassifier()

# 파이프라인으로 스태킹 모델 생성
stacking_model = Pipeline([
    # 첫 번째 레벨 모델들을 튜플로 전달하여 스태킹 모델 생성
    ('stacked_models', models_first_level),
    # 메타 모델로 LightGBM 사용
    ('meta_model', meta_model)
])

# 정확도와 F1 점수를 저장할 리스트 초기화
accuracy_scores = []
f1_scores = []

# 스태킹에 사용할 빈 배열 생성 (훈련 데이터의 예측 결과를 저장할 것임)
X_stacked_train = np.zeros((X_train.shape[0], len(models_first_level)))

# 7-kfold 스태킹 수행 (수정)
for train_idx, val_idx in kf.split(X_train):
    X_train_fold, X_val_fold = X_train[train_idx], X_train[val_idx]
    y_train_fold, y_val_fold = y_train[train_idx], y_train[val_idx]

    # 각 모델별로 학습 및 예측 수행하여 트레인 데이터를 예측 결과로 저장
    for i, (_, model) in enumerate(models_first_level):
        model.fit(X_train_fold, y_train_fold)
        X_stacked_train[val_idx, i] = model.predict(X_val_fold)

    # 메타 모델 학습
    meta_model.fit(X_stacked_train, y_train)
    
    # 검증 데이터를 사용하여 예측
    y_pred = meta_model.predict(X_stacked_train[val_idx])
    
    # 정확도와 F1 점수 계산하여 리스트에 추가
    accuracy = accuracy_score(y_val_fold, y_pred)
    f1 = f1_score(y_val_fold, y_pred)
    accuracy_scores.append(accuracy)
    f1_scores.append(f1)

# 최종 정확도 및 F1 점수 계산
mean_accuracy = np.mean(accuracy_scores)
mean_f1 = np.mean(f1_scores)

print(f"평균 정확도: {mean_accuracy:.4f}")
print(f"평균 F1 점수: {mean_f1:.4f}")


평균 정확도: 0.8462
평균 F1 점수: 0.9017


- 원본 훈련 데이터를 여러 개의 모델에 입력으로 주어 1단계 예측을 수행합니다. 이 때, 훈련 데이터를 여러 개의 폴드로 나누어 교차 검증(Cross-validation)을 사용하여 각 모델의 예측 결과를 얻습니다.

- 1단계 예측 결과를 모아 새로운 훈련 데이터를 생성합니다. 즉, 각 모델의 예측 결과를 새로운 피처로 사용합니다.

- 새로운 훈련 데이터를 이용하여 메타 모델(Meta Model)을 학습시킵니다. 메타 모델은 원본 훈련 데이터의 레이블을 타겟으로 하여 학습합니다.

- 테스트 데이터를 1단계 모델에 입력으로 주어 예측 결과를 얻습니다.

- 1단계 예측 결과를 새로운 피처로 사용하여 메타 모델을 이용하여 최종 예측을 수행합니다.

- 최종 예측 결과를 평가합니다.


제공해주신 코드: 각 모델별로 따로 학습하여 예측한 결과를 사용하여 스태킹을 수행하는 방식입니다. 먼저, 데이터를 여러 개의 폴드로 나누고, 각 폴드에 대해 각 모델을 따로 학습시킵니다. 그런 다음, 각 모델별로 예측한 결과를 저장합니다. 즉, SVM 모델의 예측 결과, 랜덤 포레스트 모델의 예측 결과, 로지스틱 회귀 모델의 예측 결과를 따로 저장합니다. 그리고 이렇게 따로 저장된 모델별 예측 결과들을 합쳐서 하나의 스태킹 모델의 입력 데이터로 사용합니다. 이렇게 합쳐진 입력 데이터로 메타 모델(여기서는 LightGBM)을 학습시킵니다.

제가 제공한 파이프라인 코드: 파이프라인을 사용하여 각 모델을 연결하고, 하나의 스태킹 모델로 학습 및 예측을 진행하는 방식입니다. 파이프라인은 여러 단계의 변환(또는 모델 학습)을 연속적으로 적용하고, 마지막 단계의 출력을 사용하여 최종 결과를 만들어냅니다. 이 방법에서는 파이프라인을 사용하여 SVM, 랜덤 포레스트, 로지스틱 회귀 세 모델을 연결하여 하나의 스태킹 모델을 만듭니다. 파이프라인에 학습 데이터를 입력하면, 각 모델이 순차적으로 학습하고 예측 결과를 합쳐서 스태킹 모델에 입력으로 사용합니다. 그리고 이렇게 하나의 스태킹 모델로 학습된 모델을 사용하여 최종 예측을 수행합니다.

두 코드 모두 스태킹 모델을 구성하는 데에 사용될 수 있고, 결과적으로 예측 결과를 얻을 수 있습니다. 다만, 제공해주신 코드는 각 모델별로 따로 예측 결과를 저장하고 합치는 작업을 직접 구현한 것이고, 파이프라인 코드는 사이킷런의 Pipeline을 사용하여 코드를 더 간결하게 구성한 것입니다. 코드를 더 간결하고 관리하기 쉽게 만들어주는 파이프라인 방법을 사용하시면 됩니다. 원하는 방식으로 코드를 선택하여 사용하시면 됩니다.